# Introduction

This is my notebook for the [2017 Advent of Code](http://adventofcode.com/2017/). I see this as an opportunity to write and share my thought processes, both for myself and anyone who comes across this (my students, hopefully). I also hope to note what skills are required for each problem, for my students and for my own memory. I remember students being concerned that these problems were outside their skill level, and I want to be able to directly point to the skills that I used to solve the problem and associate them with particular courses where possible.

Last year, I did not really attempt to compete, beyond attempting to complete all 25 days. However, based on my own casual timing with a stopwatch, I felt that I could have appeared in the rankings, so I am going to try to solve each problem this year at Midnight EST. This is typically past my bedtime, so we will see how this goes. I will be using Python throughout. I once considered Java to be my primary language, but I'm growing more comfortable with Python and I think Python is better suited to races against the clock (at least for writing code -- I know Java will actually solve the problem faster).

To begin with, I have included my solution template below. I built up this template over the course of the 2016 version, and after one day, I'm already finding things to change. Still, this will be my starting point:

In [ ]:
import itertools
import copy
import collections
import heapq
import math
import hashlib

f = open('input\\input__.txt', 'r')
lines = [line.strip() for line in f.readlines()]
f.close()

I used each of these modules multiple times. In fact, last year's Advent of Code introduced me, through colleagues and students, to the `itertools` module, which I find myself using more and more on challenges like these or Project Euler.

Inspired by Peter Norvig's template, I suspect I will add a few helper functions over the weekend. Most likely, an A* implementation is on the way, as I used that a few times last year. I suspect I will revise this template over time, but I will make a note in future days. Going deeper, this writeup in a Jupyter Notebook is also inspired by [Peter Norvig's notebook](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb), who I suspect will have better things to say about these problems than I will.

# [Day 1](http://adventofcode.com/2017/day/1): Inverse Captcha

This problem requires us to consider a single line of input -- a sequence of digits -- and find the sum of all digits that match the next digit in the sequence. That is, if the sequence includes `...55...`, then I should add 5 to my total to indicate the pair of 5s. If the sequence included `...555...`, I would add 5 twice, once for each adjacent pair. The problem also notes that we should consider the last digit of the sequence to be adjacent to the first digit of the sequence.

This is well within the capabilities of **Introduction to Programming** students. Specifically, my solution makes use of the *accumulator pattern*, *string indexing*, and *type conversion*. My solution is below:

In [1]:
def day1():
    f = open('input\\input1.txt', 'r')
    line = f.readline().strip()
    f.close()

    total = 0
    for idx in range(len(line)):
        if line[idx] == line[idx-1]:
            total += int(line[idx])

    return total

day1()

1089

This strategy takes advantage of one of my favorite, but simple, features of Python: negative indices. Instead of looking at the next digit in the sequence, my code actually checks the previous digit in the sequence. Same idea, but it means that when I start the loop at the first index of the sequence, I'm comparing to the digit at index -1 of the string. In other words, the first comparison happens between the first and last characters of the string, taking care of the tricky part of the problem right off the bat. From there, we compare each digit to the one that came immediately before it. When we have a match, we convert the digit to an integer and add it to our accumulator, `total`.

In **part two** of this puzzle, we no longer want to compare adjacent digits. Instead, we are told to compare each digit to the digit halfway through the sequence, looping back to the beginning if necessary. A useful provided sample input is `1212`. Each digit has a match. Looking at the first digit, we will have to compare to the third digit (since there are 4 digits total, we add 2 to our current comparison). These are a match. The same is true for the second digit. So our running total at this point is `1 + 2 = 3`.

Here is where I messed up, and it cost me a place in the rankings in part two: We have to compare **every** digit. So in our example from above, the third digit matches the fifth digit (which is just the first digit after we loop around). Same for the the fourth digit. So our final total is `1+2+1+2 = 6`. If we match a digit in the first half of the sequence, then we _must_ match the corresponding digit in the second half. But in my initial attempt, I assumed that we had already accounted for that. So my solution was incorrect, giving me half of the expected total. (The universe smacked me down again by having the power go out right after I got the wrong answer. So I blame not getting points in part two partially on my own misunderstanding, and partially on the power going out and distracting me. I ended up submitting my part two solution on my phone.)

My part two solution is below, which is based on my part one code:

In [2]:
def day1b():
    f = open('input\\input1.txt', 'r')
    line = f.readline().strip()
    f.close()

    length = len(line)
    half = length // 2

    total = 0
    for idx in range(len(line)):
        if line[idx] == line[(idx+half)%length]:
            total += int(line[idx])

    return total

day1b()

1156

The major change to this code comes in line 10, changing the right side of the comparison. Had I thought it through, I could have made use of negative indices, but modulo division came to mind first. Instead of comparing each index to the one before it, we compare the index to the one halfway through the sequence (represented by `idx+half`), modding by the length of the string to assure that we don't overshoot the end of the string.

# [Day 2](http://adventofcode.com/2017/day/2): Corruption Checksum

For **part one**, we are given a spreadsheet of integers. Our job is to find the difference between the minimum and maximum values in each row, and report the sum of the differences.

Again, this is solvable for any of my **Introduction to Programming** students, though my solution below uses some syntactic  techniques that I would not expect from CS1 students. Besides *file input*, the major techniques we need here are *type conversion*, *list iteration*, and the *accumulator pattern*.

My solution to part one is below:

In [3]:
def day2():
	f = open('input\\input2.txt', 'r')
	lines = [line.strip() for line in f.readlines()]
	f.close()

	total = 0
	for line in lines:
		data = [int(x) for x in line.split()]
		diff = max(data) - min(data)
		total += diff
		
	return total
	
	
day2()

47136

The list comprehension in Line 8 in this program does a lot of work. It divides each line at whitespace and converts each resulting string into an integer. So for each row, `data` is a list of integers in that row of the spreadsheet rather than a list of strings. Now that we have a list of integers, we can use `max()` and `min()` to painlessly calculate the difference between those two values.

For **part two**, we add a new wrinkle to the problem. Instead of locating the min and max in each row, we are told that there is one pair of values in each row such that one value evenly divides the other value of the pair. Once we find that pair, we should do the division and add that to our total.

My solution to this part is below, and it only adds some nested loops. It is not the cleanest solution, and I will address that below.

In [4]:
def day2b():
	f = open('input\\input2.txt', 'r')
	lines = [line.strip() for line in f.readlines()]
	f.close()

	total = 0
	for line in lines:
		data = [int(x) for x in line.split()]
		for i in range(len(data)):
			for j in range(i+1,len(data)):
				if data[i]%data[j] == 0:
					total += data[i]//data[j]
				elif data[j]%data[i] == 0:
					total += data[j]//data[i]
		
	return total

day2b()

250

The two inner `for` loops check each pair of values in a row to see if they divide evenly. If they do, we add the quotient to `total`.

Still, I had the `itertools` module in my template, but in my rush to solve the problem quickly, I ignored the module. Here is a slightly neater solution using the `combinations` function in `itertools`.

In [1]:
import itertools

def day2b_v2():
    f = open('input\\input2.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()

    total = 0
    for line in lines:
        data = [int(x) for x in line.split()]
        for i,j in itertools.combinations(data,2):
            if i%j == 0:
                total += i//j
            elif j%i == 0:
                total += j//i

    return total

day2b_v2()

250

# [Day 3](http://adventofcode.com/2017/day/3): Spiral Memory

Given a spiral strategy used to store data, identify the number of Manhattan-distance steps required to get from position 1 to the position represented by your input.

If you've done some problems in [Project Euler](https://projecteuler.net/), you've probably seen this spiral before. For my **part one** solution, I took advantage of a nice trait about the spiral. Here's the sample spiral we're given, filled out a few more steps:

`17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23  24  25`

Starting from the center (1) and working diagonally downward and to the right, we see the odd squares: 1, 9, 25. This pattern continues as the spiral grows. My strategy takes advantage of this, by finding the odd square that is closest to my input without going over. From there, we can identify it's location in the grid and continue filling out the spiral until we reach my input. Here is my solution, though as I note below, it's broken.

In [25]:
import math

def day3a(data):
    root = int(math.sqrt(data))
    loc = [root//2,-(root//2)]
    value = root**2 + 1
    height = -loc[1] + 1
    
    while value < data and loc[1]<height:
        loc[1] += 1
        value += 1
    while value < data and loc[0]>-height:
        loc[0] -= 1
        value += 1
    while value < data and loc[1]>-height:
        loc[1] -= 1
        value += 1
    while value < data and loc[0]<height:
        loc[0] += 1
        value += 1
    #print(loc)
    return (abs(loc[0])+abs(loc[1]))
    
day3a(265149)

438

Except my code doesn't exactly do what I said. It doesn't look for the nearest *odd* square -- it grabs the closest square period. For my input, that square just happens to be odd. As a result, there are other bugs in this solution. It doesn't pass the test inputs provided in the problem.

In [28]:
print(day3a(1) == 0)
print(day3a(12) == 3)
print(day3a(23) == 2)
print(day3a(1024) == 31)

True
False
False
False


In [32]:
import math

def day3a_v2(data):
    root = int(math.sqrt(data))
    if root % 2 == 0:
        root -= 1
    loc = [root//2,-(root//2)] # We're in the lower right corner of our grid.
    value = root**2
    max_xy = root//2 
    
    while value < data:
        # We need to expand the sprial now
        max_xy += 1
        
        # Let's take that next one step to the right to add root+1 to the spiral
        loc[0] += 1
        value += 1

        # From there we have to go up
        while value < data and loc[1]<max_xy:
            loc[1] += 1
            value += 1
        # Can't go up anymore, go left
        while value < data and loc[0]>-max_xy:
            loc[0] -= 1
            value += 1
        # Can't go left anymore, go down
        while value < data and loc[1]>-max_xy:
            loc[1] -= 1
            value += 1
        # Can't go down anymore, go right
        while value < data and loc[0]<max_xy:
            loc[0] += 1
            value += 1
        
        # Can't go right anymore. If we need to keep going, we need to loop back and expand.
    
    return abs(loc[0])+abs(loc[1])

print(day3a_v2(1))
print(day3a_v2(12))
print(day3a_v2(23))
print(day3a_v2(1024))
day3a_v2(265149)

0
3
2
31


438

This solution actually passes all of our test cases. Now we actually find the greatest odd square that is less than our data. Using the idea that 1 is at (0,0), we can keep track of our coordinates as we build the spiral from that odd square. To calculate the Manhattan distance, all we need to do is take the absolute value of our coordinates.

In terms of difficulty, this one kicked my butt a little bit. That said, nothing in my solution is outside of **Introduction to Programming.** I have some *nested loops*, and I am using some built-in functions, but there is nothing too advanced in this code.

The same is not true for **part two**, and I do not particularly love my solution. For one thing, I cannot use my part one solution to address this new wrinkle to the problem. For this part, we fill the grid by taking the sum of all adjacent spaces in the grid. My solution requires us to build the grid from scratch. Our goal is to identify the first number added to the grid that is larger than our input.

Let's look at this code in parts. First, here is my calcsum() function, which calculates the sum of the 8 adjacent neighbors, assuming that unfilled spaces have zeroes in them.

In [33]:
def calcsum(matrix, x, y):
    total = 0
    if x>=1 and y >= 1:
        total += matrix[x-1][y-1]
    if x>=1:
        total += matrix[x-1][y]
    if x>=1 and y<len(matrix)-1:
        total += matrix[x-1][y+1]
    if y >= 1:
        total += matrix[x][y-1]
    if y<len(matrix)-1:
        total += matrix[x][y+1]
    if x<len(matrix)-1 and y>=1:
        total += matrix[x+1][y-1]
    if x<len(matrix)-1:
        total += matrix[x+1][y]
    if x<len(matrix)-1 and y<len(matrix)-1:
        total += matrix[x+1][y+1]
    return total

My second function is the expand() function. It takes an NxN matrix and converts it to (N+2)x(N+2) by padding the entire outside with zeroes.

In [36]:
def expand(matrix):
    matlen = len(matrix)
    matrix.insert(0,[0]*(matlen+2))
    matrix.append([0]*(matlen+2))
    for row in range(1,matlen+1):
        matrix[row].insert(0,0)
        matrix[row].append(0)

Finally, the function below does all of the work.

In [43]:
def day3b(data):
    value = 1
    matrix = [[1]]
    expand(matrix)
    coords = [1,1]
    direction = 'R'
    
    while value<data:
        if direction == 'R':
            coords[0] += 1
            if coords[1] == len(matrix)-1 and coords[0] == len(matrix)-1:
                expand(matrix)
                coords[0]+=1
                coords[1]+=1
            elif coords[0] >= len(matrix)-1:
                direction = 'U'
        elif direction == 'L':
            coords[0] -= 1
            if coords[0] == 0:
                direction = 'D'
        elif direction == 'U':
            coords[1] -= 1
            if coords[1] == 0:
                direction = 'L'
        elif direction == 'D':
            coords[1] += 1
            if coords[1] == len(matrix)-1:
                direction = 'R'
        
        value = calcsum(matrix, coords[0], coords[1])
        
        matrix[coords[0]][coords[1]] = value
    return value

day3b(265149)

266330

Let's start before the while loop. Our most recent value is 1, which is at the center of a 1x1 matrix. However, we're immediately out of space, so we expand out to a 3x3 matrix. Within that matrix, our value (1) is now at `matrix[1][1]`, which is signified by `coords`. We also know that our next move is to the right.

The while loop continues so long as our last value is less than our provided input. Within that loop, we determine whether our move should be Right, Left, Up, or Down. In each case, the first step is to increase either the x- or y-coordinate. We then need to determine if we've hit an edge. If we're currently going Left, Up, or Down, then when we hit an edge, we just need to change direction. However, if we're going Right, there are two possibilities. Either we've reached the bottom right corner (in which case, the matrix is full), or we've just hit a right edge. If we hit the bottom right corner, we must expand and adjust our matrix coordinates. If we hit the right edge, we just need to change directions.

Once we've determined the direction of our next step, we can calculate the sum for this step and add it to the matrix.

Personally, I consider this solution ugly and inefficient. There is surely a better way to do this than to literally build the 2-dimensional matrix. I'm no longer certain that this code is within the realm of an **Introduction to Programming** student, mostly because of the shenanigans in the expand() function. I suspect that a **Data Structures** student could produce this code, though as I said, I think better solutions exist.

# [Day 4](http://adventofcode.com/2017/day/4): High-Entropy Passphrases

Given a list of passphrases, where each passphrase consisting of multiple words, determine the number of valid passphrases. A passphrase is not valid if it contains the same word multiple times.

For my **part one** solution, I divide each passphrase up into a list containing the words separately. I then convert that list to a set. If the lengths of the list and the set are the same (since sets cannot contain duplicates, but lists can), then the password is valid.

In [44]:
def day4a():
    f = open('input\\input4.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()
    
    total = 0
    for line in lines:
        passwords = line.split()
        pass_set = set(passwords)
        if len(passwords) == len(pass_set):
            total += 1
            
    return total
    
day4a()

383

To me, this problem sits on the border of **Introduction to Programming** and **Data Structures**. While we introduce sets in Introduction to Programming, I wonder if it wouldn't take until Data Structures for students to think of converting the list to a set in order to determine if there are no duplicates. That said, this problem is solvable for students in Introduction to Programming, but their solutions would likely be less efficient. I would expect a nested loop through the list of words to check for duplicates.

In **part two**, the rules are changed slightly. Now, a passphrase cannot contain words that are anagrams of each other. My solution is below:

In [46]:
def day4b():
    f = open('input\\input4.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()
    
    total = 0
    for line in lines:
        passwords = line.split()
        sorted_words = []
        for word in passwords:
            wordlist = list(word)
            sorted_words.append(''.join(sorted(wordlist)))
        pass_set = set(sorted_words)
        if len(sorted_words) == len(pass_set):
            total += 1
            
    return total
    
day4b()

265

Unlike Day 3, I get to build on my solution to the first part -- always a good sign in Advent of Code. Before we add the words to a set, I take each word and convert it to a list of characters. Then, I sort that list, and turn the list back into a string. Now I have `sorted_words`, a list of each word sorted by its letters. I build the set out of `sorted_words` rather than the basic `passwords`. Our validity check still compares list length against set length.

I have another version below that replaces the inner for loop with a nested list comprehension. Same steps, just compacted into one line. I'm sure I sacrifice some readability. But as I tried to build the solution, I was trying to come up with a list comprehension. Unfortunately, its structure wasn't obvious to me until I had written the loop solution above.

In [47]:
def day4b_comp():
    f = open('input\\input4.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()
    
    total = 0
    for line in lines:
        passwords = line.split()
        sorted_words = [''.join(sorted(wl)) for wl in [list(x) for x in passwords]]
        pass_set = set(sorted_words)
        if len(passwords) == len(pass_set):
            total += 1
            
    return total
    
day4b_comp()

265

In terms of difficulty, I think this part is exactly on par with the first part. My solution uses some **Data Structures** knowledge, but this problem is within reach of **Introduction to Programming** students.

# [Day 5](http://adventofcode.com/2017/day/5): A Maze of Twisty Trampolines, All Alike

Both parts of this problem require us to follow a series of numbers, which represent jumps. We need to keep count of how many jumps we make (including jumps of 0 steps) before we jump outside the bounds of the list. In **part one**, any time we jump from a particular number, we increment that number by 1. Solution below, followed by thoughts about it.

In [1]:
def day5a():
    f = open('input\\input5.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()
    
    jumps = [int(x) for x in lines]
    
    i = 0
    counter = 0
    exit = len(jumps)
    
    while i < exit:
        jumps[i] += 1
        i += (jumps[i] - 1)
        counter += 1
        
    return counter

day5a()

356945

This seemed like a particularly easy challenge, and one I would have no concern about giving to **Introduction to Programming** students. I will note that, while the concepts are easy and the required knowledge is pretty low (*arrays or lists* and *iteration* will suffice), there are opportunities to screw this up if steps are not handled in the exactly correct order. In a timed competition such as Advent of Code, that could hurt people if they make one of those mistakes.

Walking through my code, after reading the input, we convert the list of strings to a list of integers. I then set markers for my current index (`i`), a jump counter, and my threshold for when I've exceeded the bounds of the list (`exit`). I will note here that my code gets away with something, and I haven't researched whether this was true for all inputs: The problem definition states that you should terminate when you go outside the bounds of the list of numbers. My code checks when you fall off the far edge of the list, but I do not terminate if the index `i` ever becomes negative. This is an especially dangerous strategy considering how Python lets us use negative indices. I could correct this by changing my while loop condition to `while i >= 0 and i < exit:`.

Inside the while loop, I increment the jump value by 1, then increment `i` by the appropriate amount, correcting for the fact that I just incremented the jump value. Changing the order of these steps or missing the correction seem like places where people rushing to solve the problem could miss the solution. I'm lucky that I didn't get burned by ignoring one way to exceed the bounds of the list.

In **part two**, we change the rules for how to alter the jump value. Now, if the value was greater than or equal to 3, we decrement by 1. Otherwise, we continue incrementing. Solution below:

In [4]:
def day5b():
    f = open('input\\input5.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()
    
    jumps = [int(x) for x in lines]
    
    i = 0
    counter = 0
    exit = len(jumps)
    
    while i < exit:
        offset = jumps[i]
        if jumps[i] >= 3:
            jumps[i] -= 1
        else:
            jumps[i] += 1
        i += offset
        counter += 1
        
    return counter

day5b()

28372145

I'm in a good position again, because I get to build off my previous solution. The changes all come in the while loop. Besides adding the if-else block to handle the increment or decrement issue, I now save what the offset should be prior to making the change. This saves me from having to correct on line 18 the way I did in part one, which would be more difficult as I would have to keep track of whether I'm adjusting from an increment or a decrement. Also, I'm once again ignoring the edge case where `i` becomes less than zero, but I'm getting away with it.

# [Day 6](http://adventofcode.com/2017/day/6): Memory Reallocation

For this problem, our input is 16 integers, representing the number of blocks held by 16 memory banks. The reallocation procedure works by finding the memory bank with the largest load, reducing that load to zero, and distributing the blocks to the subsequent banks one at a time, looping back to the beginning of the list of blocks when necessary. We repeat this process for the bank that now has the largest load. For **part one**, the question we need to answer is how many reallocations have to occur before we reach a load state that we have already seen before.

Obviously, we need to keep track of which load states we have seen already. In my solution below, `banks` holds the current load in the 16 memory banks. I create a set, `observed`, that keeps track of which bank configurations we've already seen. We need to convert `banks` from a list to a tuple before we insert it into the set, since set contents need to be immutable. (Tuples are immutable, lists are mutable.)

Thus, the main loop continues so long as the current bank configuration is not in the observed set. When it's not, we add this (obviously new) configuration to the set, and search for the bank with the largest load. The for loop does this task, though in retrospect, it would have been much easier to replace the loop and the line before it with:
`index = banks.index(max(banks))`
I was rushing for the sake of the competition, and I thought of the loop first. Using the index() method didn't cross my mind until this morning when a colleague suggested it.

The remainder of the loop sets the overloaded bank to zero, and redistributes the load to the other banks.

In [2]:
def day6a():
    f = open('input\\input6.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()
    
    banks = [int(x) for x in lines[0].split()]
    
    observed = set()
    counter = 0
    while tuple(banks) not in observed:
        observed.add(tuple(banks))
        counter += 1
        
        index = -1
        for idx in range(len(banks)):
            if banks[idx] == max(banks):
                index = idx
                break
        
        distrib = banks[index]
        banks[index] = 0
        
        while distrib>0:
            index = (index+1)%len(banks)
            banks[index]+=1
            distrib-=1
    
    return counter
    
day6a()

5042

My solution uses concepts largely from **Introduction to Programming**. I could argue that truly understanding when/how to use tuples and sets might be a **Data Structures**-level expectation. If my students are reading this, then some of them are surely shocked at my use of `break`, considering how I admonish every student who thinks of using it. As I said above, I was speeding through for the sake of the competition. The better strategy would clearly have been using list's index() method, as I described.

For **part two**, we're asked a slightly different question. How many reallocations has it been since the first time we saw whichever configuration repeated. To address this, I added a dictionary, called `when` in the solution below. The keys to `when` are the bank configurations (in tuple format), while the values are the loop counter during the iteration where I saw it. Thus, I can change the return statement at the end to subtract the counter when I last saw the repeated configuration from the counter when I saw it the second time.

In [3]:
def day6b():
    f = open('input\\input6.txt', 'r')
    lines = [line.strip() for line in f.readlines()]
    f.close()
    
    banks = [int(x) for x in lines[0].split()]
    
    observed = set()
    when = dict()
    counter = 0
    while tuple(banks) not in observed:
        observed.add(tuple(banks))
        when[tuple(banks)] = counter
        counter += 1
        
        index = -1
        for idx in range(len(banks)):
            if banks[idx] == max(banks):
                index = idx
                break
        
        distrib = banks[index]
        banks[index] = 0
        
        while distrib>0:
            index = (index+1)%len(banks)
            banks[index]+=1
            distrib-=1
    
    return counter - when[tuple(banks)]
    
print(day6b())

1086


In fact, I could have replaced the `observed` set with the `when` dictionary entirely, rather than using both of them. The only other required change would be modifying the while loop condition to: `while tuple(banks) not in when:` The dictionary would do the entire job of the set. Still, for the sake of the competition, it was easier to keep both.

If I were to clean this code up, I would make two changes:
* In both part 1 and part 2, replace the for loop with `index = banks.index(max(banks))`
* In part 2, remove any reference to `observed`, since I can do the same job with the `when` dictionary